In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# -----------------------------
# Dataset Setup
# -----------------------------
extract_dir = "./fear_dataset"
train_dir = os.path.join(extract_dir, "train")
test_dir = os.path.join(extract_dir, "test")

selected_classes = ['angry', 'fear', 'happy', 'sad', 'surprise']

IMG_SIZE = (160, 160)  # higher resolution for MobileNetV2
BATCH_SIZE = 32

# -----------------------------
# Data Generators
# -----------------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.7,1.3],
    horizontal_flip=True,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=selected_classes,
    subset='training',
    shuffle=True,
    seed=42
)

val_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=selected_classes,
    subset='validation',
    shuffle=False,
    seed=42
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=selected_classes,
    shuffle=False
)

# -----------------------------
# Class weights to handle imbalance
# -----------------------------
from sklearn.utils.class_weight import compute_class_weight

classes = train_data.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(classes), y=classes)
class_weights_dict = dict(enumerate(class_weights))

# -----------------------------
# Model Definition
# -----------------------------
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
outputs = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

# -----------------------------
# Callbacks
# -----------------------------
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# -----------------------------
# Compile Stage 1 (Frozen)
# -----------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

print("\n🔹 Stage 1: Training top layers...")
history1 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# -----------------------------
# Stage 2: Fine-tuning
# -----------------------------
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False  # freeze first 100 layers

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

print("\n🔹 Stage 2: Fine-tuning the full model...")
history2 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    class_weight=class_weights_dict,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# -----------------------------
# Evaluate on Test Data
# -----------------------------
test_loss, test_acc = model.evaluate(test_data, verbose=1)
print(f"\n✅ Test Accuracy: {test_acc*100:.2f}%")

# Save model
model.save("mobilenetv2_fear_5class_optimized.h5")
print("\n✅ Model saved as mobilenetv2_fear_5class_optimized.h5")

Found 18647 images belonging to 5 classes.
Found 4661 images belonging to 5 classes.
Found 5834 images belonging to 5 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step

🔹 Stage 1: Training top layers...


C:\Users\Keert\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 673s 1s/step - accuracy: 0.3231 - loss: 1.5720 - val_accuracy: 0.3819 - val_loss: 1.4890 - learning_rate: 0.0010
Epoch 2/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 627s 1s/step - accuracy: 0.3725 - loss: 1.4863 - val_accuracy: 0.4186 - val_loss: 1.4387 - learning_rate: 0.0010
Epoch 3/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 608s 1s/step - accuracy: 0.3844 - loss: 1.4710 - val_accuracy: 0.4151 - val_loss: 1.4452 - learning_rate: 0.0010
Epoch 4/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 1794s 3s/step - accuracy: 0.3928 - loss: 1.4662 - val_accuracy: 0.4010 - val_loss: 1.4473 - learning_rate: 0.0010
Epoch 5/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 592s 1s/step - accuracy: 0.3953 - loss: 1.4590 - val_accuracy: 0.4244 - val_loss: 1.4268 - learning_rate: 0.0010
Epoch 6/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 4022s 7s/step - accuracy: 0.3982 - loss: 1.4536 - val_accuracy: 0.4306 - val_loss: 1.4265 - learning_rate: 0.0010
Epoch 7/15
583/583 ━━━━━━━━━━━━━━━━━━━━ 666s 1s/step - accuracy: 0.4019 - loss: 


✅ Test Accuracy: 56.56%

✅ Model saved as mobilenetv2_fear_5class_optimized.h5


In [4]:
from sklearn.metrics import accuracy_score

# Predict on test data
pred_probs = model.predict(test_data)
pred_classes = np.argmax(pred_probs, axis=1)

# Get true labels
true_classes = test_data.classes  # these are actual class indices

# Compute accuracy
acc = accuracy_score(true_classes, pred_classes)
print(f"✅ Manual Accuracy: {acc*100:.2f}%")

183/183 ━━━━━━━━━━━━━━━━━━━━ 74s 380ms/step
✅ Manual Accuracy: 56.56%


In [6]:
from sklearn.metrics import classification_report
import numpy as np

# Get predictions
pred2 = model.predict(test_data)
pred_labels = np.argmax(pred2, axis=1)

# Get true labels
true_labels = test_data.classes

# Print classification report
print(classification_report(true_labels, pred_labels, target_names=selected_classes))

183/183 ━━━━━━━━━━━━━━━━━━━━ 62s 341ms/step
              precision    recall  f1-score   support

       angry       0.45      0.43      0.44       958
        fear       0.41      0.10      0.16      1024
       happy       0.81      0.75      0.78      1774
         sad       0.40      0.77      0.53      1247
    surprise       0.76      0.61      0.68       831

    accuracy                           0.57      5834
   macro avg       0.57      0.53      0.52      5834
weighted avg       0.59      0.57      0.55      5834

